Intergrated Landslide-Flood  Early Warning System

In [7]:
"""
Installation of required Python, numpy, and pandas package
pip intall numpy pandas geapandas rasterio folium plotty dash
pip install sckit-learn matplotlib seaborn request beautfulsoup4
pip install elevation xarray netcdf4 rastestats
"""

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import requests
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'geopandas'

Rwanda Climate Risk early Warning System
 
Installation and setup script

In [ ]:
## Project directory structure

def setup_project_structure():
    """ organized folder structure for the project """

    directories = [
        "data/raw",
        "data/processed",
        "data/climate",
        "data/geographic",
        "data/socioeconomic",
        "models",
        "outputs/maps",
        "outputs/reports",
        "dashboards",
        "scripts"
    ]

    for directory in directories:
        os.makedirs(directory,exist_ok=True)
        print(f"Created directory: {directory}")

In [ ]:
## Configuration settings for Rwanda
class RwandaConfig:
    """Configuration class for Rwanda-specific parameters"""

    # Geographic bounds of Rwanda in decimal degrees
    # These define the bounding box for spatial analysis and mapping
    RWANDA_BOUNDS = {
        'min_lon': 28.8617546755,  # Western-most longitude
        'max_lon': 30.8995804700,  # Eastern-most longitude
        'min_lat': -2.9389804700,  # Southern-most latitude
        'max_lat': -1.0532659800   # Northern-most latitude
        }
    
    # Administrative divisions for regional analysis
    # Organized hierarchically from country to districts
    ADMIN_LEVELS = {
        'country': 'Rwanda',
        'province': ['Kigali', 'Northern', 'Southern', 'Eastern', 'Western'],
        'district': ['Gasabo', 'Ruhango', 'Rubavu', 'Huye', 'Musanze', 'Nyamagabe', 'Nyagatare', 'Rusizi', 
                    'Kicukiro', 'Nyarugenge', 'Kayonza', 'Karongi', 'Rutsiro', 'Gicumbi', 'Burera', 
                    'Ngororero', 'Nyabihu', 'Gakenke', 'Rulindo', 'Muhanga', 'Kamonyi', 'Bugesera', 
                    'Gatsibo', 'Kirehe']
    }

    # Major urban centers for focused analysis and reporting
    MAJOR_CITIES = ['Kigali', 'Butare', 'Gisenyi', 'Cyangugu', 'Rwamagana']

    # Hazard thresholds for early warning systems
    # These values need calibration with local historical data
    RAINFALL_THRESHOLDS = {
        'landslide': 50,     # mm of rain in 24 hours that may trigger landslides
        'flood_warning': 75, # mm of rain in 24 hours that may cause flooding
        'drought_treshold': 30  # percentage of normal rainfall indicating drought conditions
    }

    # Weighting factors for landslide risk assessment model
    # Weights sum to 1.0 representing relative importance of each factor
    MODEL_PARAMS = {
        'landslide_factors': {
            'slope_weight': 0.25,            # Terrain steepness
            'rainfall_weight': 0.20,         # Precipitation intensity
            'soil_weight': 0.15,             # Soil type and characteristics
            'landuse_weight': 0.15,          # Land cover/vegetation
            'goelogy_weight': 0.15,          # Underlying rock formations
            'infrastructure_weight': 0.10    # Human-built structures
        }
    }

We have to check if all required packages are installed

In [ ]:
def check_dependencies():
    """Check if all required packages are installed"""
    # Define a list of packages required for the project
    required_packages = [
        'numpy', 'pandas', 'geopandas', 'rasterio', 'folium',
        'sklearn', 'matplotlib', 'seaborn'
    ]
    # Initialize an empty list to track missing packages
    missing_packages = []
    
    # Attempt to import each package and track failures
    for package in required_packages:
        try:
            __import__(package)  # Dynamically import the package
            print(f"✓{package} is installed.")
        except ImportError:
            # If import fails, add to missing packages list
            missing_packages.append(package)
            print(f"✗ {package} is NOT installed")

    # Provide installation instructions if any packages are missing
    if missing_packages:
        print(f"\nInstall missing packages with:")
        print(f"pip install {' '.join(missing_packages)}")
    else:
       print("\n✓ All required packages are installed!")
       
       # Return True if all packages are installed, False otherwise
       return len(missing_packages) == 0

Download Rwanda administrative boundaries

In [ ]:
def download_rwanda_boundaries():
    """Download Rwanda administrative boundaries from GADM"""
    url = "https://geodata.ucdavis.edu/gadm/gadm4.1/shp/gadm41_RWA_shp.zip"
    response = requests.get(url)
    zip_path = r'C:\Users\educa\Desktop\Climate Resialance in Rwanda\data\raw\gadm41_RWA_shp.zip'
    
    with open(zip_path, 'wb') as file:
        file.write(response.content)
    
    print(f"Downloaded Rwanda boundaries to {zip_path}")
    
    # Unzip the file
    import zipfile
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("data/raw/")
    
    print("Extracted Rwanda boundaries.")

Lets create a sample weather station data

In [ ]:
def create_sample_data():
    """ Create sample data sttructure for testing"""

    #create a sample whether station data"
    sample_stations = pd.DataFrame({
        'station_id': ['ST001', 'ST002', 'ST003','ST004','ST005'],
        'station_name': ['Kigali', 'Huye', 'Gisenyi', 'Cyangugu', 'Rwamagana'],
        'latitude': [-1.9706,-2.6067, -1.4994,-2.4843,-2],
        'longitude': [30.1394,29.7394,29.6338,28.9086,30.7677],
        'elevation': [1500, 1700, 1600, 1400, 1550],# in meters,
        'province': ['Kigali', 'Southern', 'Western', 'Western', 'Eastern']
    })

    sample_stations.to_csv(r'C:\Users\educa\Desktop\Climate Resialance in Rwanda\data\raw\weather_stations.csv', index=False)
    print("Create sample weather stations data")

    #create a sample historical events data
    sample_events = pd.DataFrame({
        'date': pd.date_range(start='2020-01-01', periods=100, freq='D'),
        'event_type': np.random.choice(['flood', 'landslide', 'drought'], size=100),
        'district': np.random.choice(RwandaConfig.ADMIN_LEVELS['district'], size=100),
        'casualties': np.random.randint(0, 10, size=100),
        'affected_population': np.random.randint(50, 1000, size=100),
        'rainfall_mm': np.random.uniform(0, 200, size=100)
    })

    sample_events.to_csv(r'C:\Users\educa\Desktop\Climate Resialance in Rwanda\data\raw\historical_events.csv', index=False)
    print("Created sample historical events data")


Main setup function

In [ ]:
def main():
    """ Main setup function """

    print("=== Rwanda Climate Risk Early Warning System Setup ===\n")

    # Step 1: Check dependencies
    if not check_dependencies():
        print("Please install the missing packages and rerun the setup.")
        return
    # Step 2: Setup project structure
    setup_project_structure()

    # Step 3: Download Rwanda boundaries
    download_rwanda_boundaries()

    # Step 4: Create sample data
    create_sample_data()

    print("\nSetup complete! You can now start developing the Climate Risk Early Warning System for Rwanda.")
if __name__ == "__main__":
    main()